In [937]:
import sys
import json

params = {'general':{'periods':['am','pm','ip','op']}}
default = {'scenario':'orchestrator','training_folder': '../..', 'params': params} # Default execution parameters
# here sceneario: demand is only use not on lambda.
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'scenario': 'orchestrator', 'training_folder': '../..', 'params': {'general': {'periods': ['am', 'pm', 'ip', 'op']}}}


In [938]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
import numpy as np

In [939]:
periods = argv['params']['general']['periods']
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)
training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'
if on_lambda:
    bucket_name = 'quetzal-immense' #os.environ.get('BUCKET_NAME')
    output_folder = training_folder + '/outputs/'
    scenario_folders = []
    input_folders = []
    for p in periods:
        folder = 's3://' + bucket_name + '/' + p + '/outputs/'
        scenario_folders.append(folder)
        input_folders.append(input_folder + p + '/')

else:
    output_folder = training_folder + '/scenarios/' + argv['scenario'] + '/outputs/'
    scenario_folders = []
    input_folders = []
    for p in periods:
        folder = training_folder + '/scenarios/' + p + '/outputs/'
        scenario_folders.append(folder)
        input_folders.append(input_folder + p + '/')
    
print('input folder: ', input_folders)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folders)

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

On Lambda :  False
input folder:  ['../../inputs/am/', '../../inputs/pm/', '../../inputs/ip/', '../../inputs/op/']
output folder:  ../../scenarios/orchestrator/outputs/
scen folder :  ['../../scenarios/am/outputs/', '../../scenarios/pm/outputs/', '../../scenarios/ip/outputs/', '../../scenarios/op/outputs/']


In [940]:
base_volumes = pd.read_csv(input_folder+'base/volumes/volumes.csv')
base_volumes.index.name = 'index'
volumes = base_volumes.copy()

In [941]:
volumes['origin'] = 'zone_' + volumes['origin'].astype(str)
volumes['destination'] = 'zone_' + volumes['destination'].astype(str)

In [942]:
class_dict = {'C_other':'car',
            'C_EB':'car',
            'BPAX':'pt', 
            'RPAX':'pt'}
volumes['mode'] = volumes['vehicle_class'].apply(class_dict.get)


In [943]:
volumes = volumes.groupby(['origin','destination','time_period','mode'])[['volume']].agg(sum).unstack().reset_index()

In [944]:
volumes.columns = ['origin','destination','time_period','car','pt']

In [945]:
volumes = volumes.fillna(0)

In [ ]:
period_dict={'am':'AM','pm':'PM','ip':'IP','op':'OP'}


In [ ]:
#pour martin
cols = ['origin', 'destination', 'time_period', 'vehicle_class', 'time']
skim = pd.DataFrame()
ref_skim = pd.DataFrame()
for period, ref_folder, scen_folder in zip(periods,input_folders,scenario_folders):
    print(period)
    ref_pt_skim = pd.read_csv(scen_folder+'ref_pt_skim.csv')
    ref_car_skim = pd.read_csv(scen_folder+'ref_car_skim.csv')
    pt_skim = pd.read_csv(scen_folder+'pt_skim.csv')
    car_skim = pd.read_csv(scen_folder+'car_skim.csv')
    
    ref_pt_skim['time_period'] = period_dict[period]
    ref_car_skim['time_period'] = period_dict[period]
    pt_skim['time_period'] = period_dict[period]
    car_skim['time_period'] = period_dict[period]

    ref_skim = pd.concat([ref_skim, ref_pt_skim[cols]])
    ref_skim = pd.concat([ref_skim, ref_car_skim[cols]])
    skim = pd.concat([skim, pt_skim[cols]])
    skim = pd.concat([skim, car_skim[cols]])
    
    # skims pour martin:
    # origin, destination, time_period, vehicle_class, time,
    # ca: pt et road dans le meme fichier (vehicle_class => pt ou road)
    # ref et pas ref.

In [924]:
skim.reset_index(drop=True).to_csv(output_folder+'skims'+'.csv')
ref_skim.reset_index(drop=True).to_csv(output_folder+'ref_skims'+'.csv')

# modal shift

In [948]:
def get_pt_part(pt_los:pd.DataFrame, 
                car_los:pd.DataFrame, 
                method:str='kirchhoff',
                power:float=-1.42,
                part_max:float=1) -> pd.DataFrame:
    
    cols = ['origin','destination','time']
    los = pt_los[cols].merge(car_los[cols],on=['origin','destination'],how='inner')
    los = los.rename(columns = {'time_x':'time_pt','time_y':'time_car'})
    los['time_ratio'] = los['time_pt'] / los['time_car']
    if method == 'kirchhoff':
        los['pt_part'] = los['time_ratio'].apply(lambda x: min(x**(power), part_max))
    else:
        print('method not recognized')
    return los


In [949]:
for period, ref_folder, scen_folder in zip(periods,input_folders,scenario_folders):
    print(period)
    ref_pt_skim = pd.read_csv(scen_folder+'ref_pt_skim.csv')
    ref_car_skim = pd.read_csv(scen_folder+'ref_car_skim.csv')
    pt_skim = pd.read_csv(scen_folder+'pt_skim.csv')
    car_skim = pd.read_csv(scen_folder+'car_skim.csv')
    
    v = volumes[volumes['time_period'] == period_dict[period]]
    
    ref_pt_part = get_pt_part(ref_pt_skim, ref_car_skim)
    pt_part = get_pt_part(pt_skim, car_skim)
    
    ref_pt_part = ref_pt_part.set_index(['origin','destination'])['pt_part'].to_dict()
    pt_part['ref_pt_part'] = pt_part.set_index(['origin','destination']).index.map(ref_pt_part.get)
    pt_part['transfert_prob'] = pt_part['pt_part'] - pt_part['ref_pt_part']
    
    transfert_prob = pt_part.set_index(['origin','destination'])['transfert_prob'].to_dict()
    v['transfert_prob'] = v.set_index(['origin','destination']).index.map(transfert_prob.get)
    v['transfert'] = v['pt'] * v['transfert_prob']
    
    volumes.loc[volumes['time_period'] == period_dict[period],'new_pt'] =v['pt'] + v['transfert']
    volumes.loc[volumes['time_period'] == period_dict[period],'new_car'] = v['car'] - v['transfert']
    
    #volumes[['origin','destination','pt','car']].to_csv(output_folder+'volume_'+period+'.csv')

am
pm
ip
op


In [950]:
volumes['new_car'] = volumes['new_car'].fillna(volumes['car'])
volumes['new_pt'] = volumes['new_pt'].fillna(volumes['pt'])

In [951]:
volumes['car_growth'] = volumes['new_car']/volumes['car']
volumes['pt_growth'] = volumes['new_pt']/volumes['pt']

In [952]:
base_volumes['o'] = 'zone_' + base_volumes['origin'].astype(str)
base_volumes['d'] = 'zone_' + base_volumes['destination'].astype(str)

In [953]:
car_dict = volumes.set_index(['origin','destination','time_period'])['car_growth'].to_dict()
pt_dict = volumes.set_index(['origin','destination','time_period'])['pt_growth'].to_dict()

In [954]:
base_volumes['pt_growth'] = base_volumes.set_index(['o','d','time_period']).index.map(pt_dict.get)
base_volumes['car_growth'] = base_volumes.set_index(['o','d','time_period']).index.map(car_dict.get)

In [955]:
car_filter = base_volumes['vehicle_class'].isin(['C_other', 'C_EB'])
base_volumes.loc[car_filter,'volume'] = base_volumes.loc[car_filter,'volume'] * base_volumes.loc[car_filter,'car_growth'] 


In [956]:
pt_filter = base_volumes['vehicle_class'].isin(['BPAX', 'RPAX'])
base_volumes.loc[pt_filter,'volume'] = base_volumes.loc[pt_filter,'volume'] * base_volumes.loc[pt_filter,'pt_growth'] 


In [957]:
base_volumes = base_volumes.drop(columns=['o','d','pt_growth','car_growth'])

In [958]:
base_volumes.to_csv(output_folder+'volumes'+'.csv')

In [961]:
end_of_notebook

NameError: name 'end_of_notebook' is not defined

# incremental

In [235]:
# 12.3.2 Incremental or Pivot-point Modelling

In [236]:
volumes = pd.read_csv(ref_folder+'volumes/volumes.csv')
#volumes = volumes.set_index('index')

ref_pt_skim = pd.read_csv(ref_folder+'skims/pt_skim.csv')
ref_car_skim = pd.read_csv(ref_folder+'skims/car_skim.csv')
pt_skim = pd.read_csv(scen_folder+'pt_skim.csv')
car_skim = pd.read_csv(scen_folder+'car_skim.csv')

pt_skim['time'] = pt_skim['time']-60

In [237]:
cols = ['origin','destination','time']

ref_los = ref_pt_skim[cols].merge(ref_car_skim[cols],on=['origin','destination'],how='inner')
ref_los = ref_los.rename(columns = {'time_x':'time_pt','time_y':'time_car'})
los = pt_skim[cols].merge(car_skim[cols],on=['origin','destination'],how='inner')
los = los.rename(columns = {'time_x':'time_pt','time_y':'time_car'})

# pt, car is utility (or time)

In [238]:
timescale = 60 # seconds to minutes
constante_modal_tc = 0
los['utility_car'] = -los['time_car']/timescale
los['utility_pt'] = -(los['time_pt']+constante_modal_tc)/timescale

ref_los['utility_car'] = -ref_los['time_car']/timescale
ref_los['utility_pt'] = -(ref_los['time_pt']+constante_modal_tc)/timescale

In [239]:
volumes['proportion_pt'] = volumes['pt']/(volumes['pt']+volumes['car'])
volumes['proportion_car'] = volumes['car']/(volumes['pt']+volumes['car'])
volumes['proportion_pt'] = volumes['proportion_pt'].fillna(0)
volumes['proportion_car'] = volumes['proportion_car'].fillna(0)

In [240]:
volumes['delU_pt'] = los['utility_pt'] - ref_los['utility_pt']
volumes['delU_car'] = los['utility_car'] - ref_los['utility_car']

In [241]:
denum = volumes['proportion_pt']*np.exp(volumes['delU_pt']) + volumes['proportion_car']*np.exp(volumes['delU_car'])
volumes['new_proportion_pt'] = volumes['proportion_pt']*np.exp(volumes['delU_pt']) / denum
volumes['new_proportion_car'] = volumes['proportion_car']*np.exp(volumes['delU_car']) / denum

volumes['new_proportion_pt'] = volumes['new_proportion_pt'].fillna(0)
volumes['new_proportion_car'] = volumes['new_proportion_car'].fillna(0)

In [579]:
volumes

,origin,destination,time_period,car,pt,new_pt,new_car
0,zone_1,zone_1,IP,0.0,0.027379,NaN,NaN
1,zone_1,zone_10,AM,0.0,17.991072,16.938099,1.052973
2,zone_1,zone_10,IP,0.0,7.142308,NaN,NaN
3,zone_1,zone_10,OP,0.0,3.262046,NaN,NaN
4,zone_1,zone_10,PM,0.0,3.589325,NaN,NaN
...,...,...,...,...,...,...,...
492143,zone_99,zone_98,AM,0.0,0.114237,0.110720,0.003517
492144,zone_99,zone_98,IP,0.0,0.005365,NaN,NaN
492145,zone_99,zone_98,OP,0.0,0.027998,NaN,NaN
492146,zone_99,zone_99,AM,0.0,0.049822,NaN,NaN


In [679]:
#check dans 12 nested logit in logic. 
#remonter lutiliter dun nid inf a sup

In [891]:
base_volumes.groupby('vehicle_class')['volume'].agg(sum)

vehicle_class
BPAX       3.410285e+04
C_EB       4.911758e+04
C_other    1.305102e+06
RPAX       2.052977e+04
Name: volume, dtype: float64